In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

In [4]:
# 실행할 때 마다 같은 값을 얻어오기 위한 tensorflow, numpy random seed 값을 설정한다.
seed = 0
tf.set_random_seed(seed)
np.random.seed(seed)

In [8]:
# 학습할 데이터를 만듭니다.
# xData : [공부시간, 과외시간]
'''
xData = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
print(xData)
print(type(xData))
'''
xData = np.array([[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]) # 7행 2열 numpy 배열
print(xData)
print(type(xData)) # <class 'numpy.ndarray'>

[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [13]:
# yData : [합격 여부] -> 실제값
'''
yData = [0, 0, 0, 1, 1, 1, 1]
print(yData)
print(type(yData)) # <class 'list'>
'''
# yData = np.array([0, 0, 0, 1, 1, 1, 1]) # 1행 7열 numpy 배열
# reshape() : numpy에서 데이터는 그대로 유지한 채 배열의 형태(차원)을 변경합니다. 1행 7열 numpy 배열 -> 7행 1열 numpy 배열
yData = np.array([0, 0, 0, 1, 1, 1, 1]).reshape(7, 1)
print(yData)
print(type(yData)) # <class 'numpy.ndarray'>

[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


In [14]:
# xData와 yData를 저장할 placeholder를 만듭니다.
# placeholder에 numpy 배열이 대입되는 경우 shape 속성을 이용해 대입되는 numpy 배열의 차원을 지정해야 합니다.
# [None, 2] -> numpy 행의 개수는 몇개라도 상관없고 열의 개수는 2개입니다.
X = tf.placeholder(dtype=tf.float64, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float64, shape=[None, 1]) # xData가 대입될 placeholder

In [15]:
# 기울기(가중치) a와 y절편(바이어스) b의 값을 임의(random)로 정합니다.
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float64)) # 난수를 2행 1열로 발생시킵니다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float64))

In [16]:
# 시그모이드 방정식을 만듭니다.
# tf.matmul(X, a) : tensorflow에서 행렬의 곱셈을 계산하는 함수 -> 행렬 X([공부시간, 과외시간])에 행렬 a(가중치)를 곱합니다.
# tf.sigmoid() : tensorflow에서 시그모이드 방정식을 계산하는 함수
y = tf.sigmoid(tf.matmul(X, a) + b)

# 오차 함수
loss = -tf.reduce_mean(Y * tf.log(y) + (1-Y) * tf.log(1-y))
# 경사 하강 알고리즘을 이용해서 오차 함수의 실행 결과를 최소로 하는 값을 찾도록 하는 식을 만듭니다.
gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss)


In [18]:
# sigmoid() 함수의 실행 결과(y) 값이 0.5 이상이면 1을 0.5 미만이면 0을 리턴시킵니다.
# tf.cast(캐스팅 할 데이터, dtype=캐스팅 될 데이터 타입) : tensorflow에서 형변환을 실행하는 함수
# predicted = tf.cast(tf.constant([1.9, 2.1]), tf.int32) # [1, 2] -> 실수(부동 소수점 데이터)를 정수형 데이터로 변환합니다.
# cast() 함수의 캐스팅 할 데이터가 boolean 타입이면 True는 1로 False는 0으로 캐스팅합니다.
# predicted = tf.cast(0.4 >= 0.5, dtype=tf.float64) # 조건 비교 결과가 False이므로 0.0으로 캐스팅 됩니다.
# predicted = tf.cast(0.4 <= 0.5, dtype=tf.float64) # 조건 비교 결과가 True이므로 1.0으로 캐스팅 됩니다.

predicted = tf.cast(y >= 0.5, dtype = tf.float64) # 예상값, sigmoid() 함수의 실행 결과(y)를 0과 1로 기억시킵니다.

# 예상값이 실제값과 일치하는 정확도를 계산합니다.
# tf.eaual() : tensorflow에서 인수로 지정된 값이 같으면 True, 다르면 False를 리턴하는 함수
# equal() 함수로 예상값(predicted)과 실제값(Y)이 같은지 비교 후 True 또는 False 결과를 tf.float64로 캐스팅 시키고 전체 평균을 계산합니다.

accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float64)) # 정확도

In [29]:
# 학습시킵니다.
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # tensorflow 변수를 초기화 시킵니다.
    for i in range(20001):
        # sess.run([기울기, y절편, 오차함수, 경사하강], feed_dict = {X: 입력데이터, Y: 실제값})
        a_, b_, loss_, _ = sess.run([a, b, loss, gradient_descent], feed_dict = {X: xData, Y: yData})
        '''
        if i % 300 == 0:
            # Epoch : 학습횟수, a1 : 기울기(공부시간), a2 : 기울기(과외시간), b : y절편, loss : 오차함수
            print('Epoch = {}, a1 = {}, a2 = {}, b = {}, loss = {}'.format(i, a_[0], a_[1], b_, loss_))
        '''
# 학습 완료 ==========================================================================================
    # 테스트 데이터를 만듭니다.
    # new_x = np.array([6, 5]) # 테스트 데이터가 1차원 numpy 배열이기 때문에 행렬의 곱셈 연산이 실행되지 않습니다.
    # print(new_x)
    new_x = np.array([6,5]).reshape(1, 2) # reshape() 함수를 이용해서 1행 2열인 2차원 numpy 배열을 만듭니다.
    # print(new_x)

    # sigmoid() 함수 연산 결과에 테스트 데이터를 대입시켜 연산합니다.
    result, new_y = sess.run([predicted, y], feed_dict = {X: new_x}) # result : 합격여부, new_y : 합격 확률
    print('공부 시간 : %d, 과외 시간 : %d' % (new_x[:, 0], new_x[:, 1]))
    print('합격 여부(sigmoid() 함수 실행 결과를 0.5 이상은 1, 미만은 0으로 변환합니다.) : %d, 합격 확률(sigmoid() 함수 실행 결과) : %f' % (result, new_y))

    for i in range(11):
        new_x = np.array([0, i]).reshape(1, 2)
        result, new_y = sess.run([predicted, y], feed_dict={X: new_x})
        print('공부 시간 : %d, 과외 시간 : %2d' % (new_x[:, 0], new_x[:, 1]), end = ', ')
        print('합격 여부 : %s, 합격 확률 : %8.4f%%' % (' 합격' if result == 1 else '불합격', new_y * 100))

공부 시간 : 6, 과외 시간 : 5
합격 여부(sigmoid() 함수 실행 결과를 0.5 이상은 1, 미만은 0으로 변환합니다.) : 1, 합격 확률(sigmoid() 함수 실행 결과) : 0.684702
공부 시간 : 0, 과외 시간 :  0, 합격 여부 : 불합격, 합격 확률 :   0.0000%
공부 시간 : 0, 과외 시간 :  1, 합격 여부 : 불합격, 합격 확률 :   0.0005%
공부 시간 : 0, 과외 시간 :  2, 합격 여부 : 불합격, 합격 확률 :   0.0357%
공부 시간 : 0, 과외 시간 :  3, 합격 여부 : 불합격, 합격 확률 :   2.2824%
공부 시간 : 0, 과외 시간 :  4, 합격 여부 :  합격, 합격 확률 :  60.4616%
공부 시간 : 0, 과외 시간 :  5, 합격 여부 :  합격, 합격 확률 :  99.0110%
공부 시간 : 0, 과외 시간 :  6, 합격 여부 :  합격, 합격 확률 :  99.9847%
공부 시간 : 0, 과외 시간 :  7, 합격 여부 :  합격, 합격 확률 :  99.9998%
공부 시간 : 0, 과외 시간 :  8, 합격 여부 :  합격, 합격 확률 : 100.0000%
공부 시간 : 0, 과외 시간 :  9, 합격 여부 :  합격, 합격 확률 : 100.0000%
공부 시간 : 0, 과외 시간 : 10, 합격 여부 :  합격, 합격 확률 : 100.0000%


[[6 5]]


RuntimeError: Attempted to use a closed Session.